In [1]:
# import PyTorch and its related packages
import torch as T
import pandas as pd
import hepytorch as hep
import matplotlib.pyplot as plt

In [2]:
num_inputs = 2
num_outputs = 1
num_examples = 10000
dtype = T.float

device = 'cuda' if T.cuda.is_available() else 'cpu'

def real_fn(X):
    return 2 * X[:, 0] - 3.4 * X[:, 1] + 4.2

X = T.randn(num_examples, num_inputs, device=device, dtype=dtype)
noise = .1 * T.randn(num_examples, device=device, dtype=dtype)
Y = (real_fn(X) + noise).view(-1, 1)

In [ ]:
plt.scatter(X[:, 1].cpu().numpy(), Y.cpu().numpy())
plt.show()

In [ ]:
plt.scatter(X[:, 0].cpu().numpy(), Y.cpu().numpy())
plt.show()

In [ ]:
# define dataframe for the dataset
x = X.cpu().numpy()
y = Y.cpu().numpy()
x.shape, y.shape
df = pd.DataFrame({'x1': x[:, 0], 'x2': x[:, 1], 'y': y[:, 0]})
df.to_csv('./data/linear_regression_data.csv', index=False)
df.head()

In [2]:
myHEP = hep.HEPTorch('./recipes/linear_regression.json')

In [3]:
result = myHEP.train()

Epoch 0, loss: 2.8999868668079376
Epoch 1, loss: 1.9189559776306153
Epoch 2, loss: 1.2698826123476028
Epoch 3, loss: 0.8404918667554855
Epoch 4, loss: 0.5564154411137104
Epoch 5, loss: 0.3684860820412636
Epoch 6, loss: 0.24416069824695588
Epoch 7, loss: 0.16189673876464367
Epoch 8, loss: 0.10746655955314637
Epoch 9, loss: 0.07145826536566019
Epoch 10, loss: 0.047631600704044104
Epoch 11, loss: 0.031864122223109005
Epoch 12, loss: 0.021431478307768703
Epoch 13, loss: 0.014527877839282156
Epoch 14, loss: 0.009959215485304594
Epoch 15, loss: 0.006935682286135852
Epoch 16, loss: 0.004934495775587857
Epoch 17, loss: 0.0036102711639367045
Epoch 18, loss: 0.0027342130046803504
Epoch 19, loss: 0.002153906250046566
Epoch 20, loss: 0.0017698166421847418
Epoch 21, loss: 0.00151545222888235
Epoch 22, loss: 0.001347083861939609
Epoch 23, loss: 0.0012355925934389234
Epoch 24, loss: 0.0011617398370523005
Epoch 25, loss: 0.0011128997777355835
Epoch 26, loss: 0.0010806018928880804
Epoch 27, loss: 0.001

In [ ]:
losses = result['losses']
print(losses)

In [ ]:
params = myHEP.model.parameters()
print('The type of "params" is a ', type(params))

for name, param in myHEP.model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

In [ ]:
############################################
#    Script to plot the losses over time
############################################
def plot(model, losses, X, sample_size=100):
    xs = list(range(len(losses)))
    f, (fg1, fg2) = plt.subplots(1, 2)
    fg1.set_title('Loss during training')
    fg1.plot(xs, losses, '-r')
    fg2.set_title('Estimated vs real function')
    fg2.plot(X[:sample_size].cpu().detach().numpy(),
             model(X[:sample_size, :]).cpu().detach().numpy(), 'or', label='Estimated')
    fg2.plot(X[:sample_size].cpu().detach().numpy(),
             real_fn(X[:sample_size, :]).cpu().detach().numpy(), '+g', label='Real')
    fg2.legend()

    plt.show()

            
plot( myHEP.model, losses, X, 100)
